This Notebook is the official version of the EDA code which includes the following 

<li>Loading of the libraries and Amazon Dataframe
<li>Dataset Statistics
<li>Pre-Processing 
<li>Sentiment Analysis
<li>EDA 
<li>Export to files (for Vizualisation in Tableau)
</li>

Sampling - Cmd 13 requires modification based on the size of the sample that is required for the EDA. Default is 0.40.

In [2]:
#Import SPARK NLP and NLTK print version
import sparknlp
print("Spark NLP version")
sparknlp.version()

import nltk
nltk.download('stopwords')

#Import relevant packages for pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from nltk.corpus import stopwords


import pyspark.sql.functions as f
from pyspark.sql.functions import length, when

In [3]:
# Load in one of the tables
df1 = spark.sql("select 'games' as type, * from default.video_games_5")
df2 = spark.sql("select 'books' as type,* from default.books_5_small")
df3 = spark.sql("select 'home' as type,* from default.home_and_kitchen_5_small")

#Add dataset label to determine if its a book, videogame or home review (Nic+Jessee)

df_complete = df1.union(df2).union(df3)
print((df_complete.count(), len(df_complete.columns)))

(3487331, 13)

###Dataset Statistics

In [5]:
#Review per category
df_complete.groupBy('type').count().show()

+-----+-------+
 type| count|
+-----+-------+
games| 487419|
books| 999913|
 home|1999999|
+-----+-------+

In [6]:
#Count of reviews per rating
display(df_complete.groupBy('overall').count())

overall,count
1.0,179046
4.0,612575
3.0,291621
2.0,153010
5.0,2251079


In [7]:
#Distinct ReviewerID
from pyspark.sql.functions import countDistinct
df_complete.agg(countDistinct("reviewerID")).show()


+--------------------------+
count(DISTINCT reviewerID)|
+--------------------------+
 1132561|
+--------------------------+

In [8]:
#Number of unique Items
from pyspark.sql.functions import countDistinct
df_complete.agg(countDistinct("asin")).show()

+--------------------+
count(DISTINCT asin)|
+--------------------+
 65504|
+--------------------+

In [9]:
#Most common review Name
from pyspark.sql.functions import desc
df_complete.groupBy("asin").count().sort(desc("count")).show()

+----------+-----+
 asin|count|
+----------+-----+
0007420412|19655|
000711835X|18496|
0007548672|15921|
0007350899| 7890|
0007444117| 7830|
B000YGEVMI| 7356|
0007378033| 6336|
B0015TMHSI| 5846|
006017322X| 5161|
0007384289| 5095|
0007350783| 4803|
0002247399| 4598|
0007155662| 4575|
0007141424| 4493|
B0014CX87U| 4440|
0008220565| 4252|
0007271239| 3959|
0060740221| 3922|
B00164M9XA| 3806|
000755236X| 3668|
+----------+-----+
only showing top 20 rows

In [10]:
#Most common review Name
from pyspark.sql.functions import desc
df_complete.groupBy("ReviewerName").count().sort(desc("count")).show()

+---------------+------+
 ReviewerName| count|
+---------------+------+
Amazon Customer|129946|
Kindle Customer| 31460|
 Chris| 3263|
 Mike| 3057|
 John| 2864|
 Linda| 2411|
 Michael| 2391|
 David| 2278|
 Mary| 2047|
 Jennifer| 2020|
 Susan| 1959|
 Sarah| 1905|
 Karen| 1835|
 Alex| 1809|
 Steve| 1803|
 Lisa| 1750|
 Anonymous| 1710|
 Brian| 1687|
 Amanda| 1646|
 James| 1643|
+---------------+------+
only showing top 20 rows

In [11]:
#Reviewer with highest count of review
from pyspark.sql.functions import desc
display(df_complete.groupBy("reviewerID").count().sort(desc("count")))

In [12]:
#Display the reviewer with the highest ammount of reviews.
display(df_complete.filter("ReviewerID = 'A3V6Z4RCDGRC44'").sort(desc("reviewTime")))

type reviewID overall vote verified reviewTime reviewerID asin reviewerName reviewText summary unixReviewTime label games 51169 5.0 10 false 12 7, 2003 A3V6Z4RCDGRC44 B0000CC7H5 Lisa Shea In this puzzle-adventure set in an Egypt-like location, you switch between two characters. One is Sphinx, a tail-toting teenage guy, and the other is a mummy. The graphics in the game are great. Your tail swishes from side to side, the wooden bridges sag beneath your weight, your shadows move along with you. It has a cartoony feel to it, which is fine for its style. The smoke and fire aren't very well done, but those are always tricky. While sound really shouldn't be a key for a game, it is sadly lacking in this one. The background music and ambient noises are OK, usually very faint. But this game has a ton of dialogue and there is no speaking! That might have been fine back in the days of Final Fantasy 8, but in this day and age it really feels lacking every time an important dialogue is going on and there is just ... silence. The actual gameplay is very enjoyable. The characters' movements are smooth and natural, from swinging up a rope to leaping across chasms. Each map has enough area for you to explore and not feel stuck, but it's not so huge that you get lost. There are good assortments of puzzles and action sequences to keep both your fingers and your brain engaged. Sure, a few of the puzzles are a bit hokey - but that's the case in any game. The save feature isn't great, but you learn to deal with that. In general it's a fun game for all ages! Great graphics and interesting gameplay 1070755200 1 games 36983 5.0 2 true 12 7, 2003 A3V6Z4RCDGRC44 B00006F2ZQ Lisa Shea The Metal Gear Solid series is famous for its intense gameplay. Metal Gear Solid 2 adds to the series with combat, stealth, strategy and intrigue. You are Raiden / Snake, part of a covert military group. The graphics in this game are just amazing. The movements, surroundings, lighting are very well done. You can peer around corners, crouch down, crawl along to sneak up up enemies. The realism isn't quite as nice as some other games out there - it's not "gritty" - but it's close. The sounds fit in well with the game. The background music is moody and suspense-filled, and the ambient noises of footfalls and doors opening and closing all reallyd raw you into the game. The gameplay is great, in that it's not a blast-em-up button masher. You have to think about your surroundings and work to take out your enemies in the most efficient way you can. You can replay levels to find new and better ways to get through the level. Where the game drags is in the PLOT. First off, the plot is more like a soap opera, with all sorts of really bizarre twists and interpersonal conflicts. Second, this is supposed to be a stealth-trained guy, a la splinter cell. Instead, he gets these INCREDIBLY long discussions while he's in the middle of a mission! They should have had briefing sessions BEFORE each mission that you could skip, and then when you were actually in the mission you should just do it. There could maybe be an "on line encyclopedia" you could go to if you got stuck - but to keep deliberately interrupting the game for these dialogues was poor. Still, you can always have a book nearby to read while those dialogues go on - I hate to skip them just in case something important IS said in them. Usually nothing is, but you never know. If you can last through those cut-scenes, the rest of the game is really enjoyable and well worth playing. It's a lesson in patience! Great gameplay but LONG cut scenes 1070755200 1 games 35647 5.0 8 false 12 7, 2003 A3V6Z4RCDGRC44 B000069BCN Lisa Shea Hitman 2 is a first person shooter which rewards control and stealth over full-out blasting. Gorgeous graphics and freeform gameplay make this great for teenagers and adults. To start out, you're a retired hitman, enjoying the beauty, peace, and serenity of Sicily, Italy. You talk with your friend, the priest, about

In [13]:
#Creates a sample of the full dataset for the EDA. Recommend using 40%.
df = df_complete.sample(fraction=0.4, seed=4) #this is the dataset without the term frequency 

In [14]:
# Drop duplicates

df = df.dropDuplicates(['reviewerID', 'asin'])


In [15]:
# Convert Unix timestamp to readable date

from pyspark.sql.functions import from_unixtime, to_date

df = df.withColumn("reviewTime", to_date(from_unixtime(df.unixReviewTime))) \
                                                .drop("unixReviewTime")

# Fill in the empty vote column with 0, and convert it to numeric type

from pyspark.sql.types import *

df = df.withColumn("vote", df.vote.cast(IntegerType())) \
                                                 .fillna(0, subset=["vote"]) 

In [16]:
from pyspark.sql import functions as sf
df_joined = df.withColumn('joined_column', 
                    sf.concat(sf.col('summary'),sf.lit(' '), sf.col('reviewText')))

#Drop Rows with Null in Joined Column
df_joined=df_joined.filter(sf.col('joined_column').isNotNull())        
#print("New number of NAs/Nulls")
newNA=df_joined.select('joined_column').withColumn('isNull_c',sf.col('joined_column').isNull()).where('isNull_c = True').count()
#print(newNA)

In [17]:

#Download stopwords dictionary
eng_stopwords = stopwords.words('english')

#Start SPARK NLP
spark = sparknlp.start()

#Make new Dataframe
df_preprocess=df_joined

#Create different components of the Pipeline steps-> reads in file -> tokenize -> removes stop words -> lemmatizes -> unigrams/ngrams ->outputs columns
document = DocumentAssembler()\
    .setInputCol("joined_column")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setSplitChars(['-']) \
    .setContextChars(['(', ')', '?', '!'])

remover = StopWordsCleaner() \
    .setInputCols(["token"])\
    .setOutputCol("nostopwords")\
    .setStopWords(eng_stopwords)

normalizer = Normalizer() \
     .setInputCols(['nostopwords']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

normalizerCaps = Normalizer() \
     .setInputCols(['nostopwords']) \
     .setOutputCol('normalized_wCaps') \
     .setLowercase(False)

stemmer = Stemmer() \
    .setInputCols(["normalized"]) \
    .setOutputCol("stem")

lemmatizer = LemmatizerModel.pretrained(name="lemma_antbnc", lang="en") \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemmat") 

ngram = NGramGenerator() \
            .setInputCols(["normalized"]) \
            .setOutputCol("ngrams") \
            .setN(2) \
            .setEnableCumulative(True)\
            .setDelimiter("_")
  
pos = PerceptronModel.pretrained('pos_anc') \
     .setInputCols(['document', 'normalized']) \
     .setOutputCol('pos')

allowed_tags = ['<JJ>+<NN>', '<NN>+<NN>'] #setting ngrams to meaningful sets for the Chunker function

chunker = Chunker() \
     .setInputCols(['document', 'pos']) \
     .setOutputCol('nmeaning') \
     .setRegexParsers(allowed_tags)

finisher = Finisher() \
     .setInputCols(['token','nostopwords','normalized','normalized_wCaps','stem','lemmat','ngrams', 'nmeaning']) #these are the columns that will be printed to the df
  
pipeline = Pipeline(
    stages = [
        document,
        tokenizer,
        remover,
        normalizer,
        normalizerCaps,
        stemmer,
        lemmatizer,
        ngram,
        pos,
        chunker,
        finisher
    ])

pipelineModel = pipeline.fit(df_preprocess)
df_preprocess = pipelineModel.transform(df_preprocess)


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]
pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[ | ][ / ][OK!]

In [18]:
import pyspark.sql.functions as f
from pyspark.sql.functions import length, when

#Count number of words
df_preprocess = df_preprocess.withColumn('wordCount', f.size(f.split(f.col('joined_column'), ' ')))
df_preprocess = df_preprocess.withColumn('wordCount_normalized', f.size(f.col('finished_normalized_wCaps')))

#Count number of char
df_preprocess = df_preprocess.withColumn('charCount', length('joined_column'))

###Most popular N-Gram over Time

In [20]:
df_preprocess.cache()
df_word_time = df_preprocess.select(f.explode('finished_nmeaning'), 'reviewTime')

In [21]:
df_word_time2 = df_word_time.filter(df_word_time.col=="non-stick")
display(df_word_time2)

In [22]:
df_word_time3 = df_word_time.filter(df_word_time.col=="easy to clean")
display(df_word_time3)

col,reviewTime
easy to clean,2017-02-26
easy to clean,2015-09-09
easy to clean,2013-10-17
easy to clean,2017-11-02
easy to clean,2014-11-05
easy to clean,2011-05-20
easy to clean,2013-11-09
easy to clean,2011-10-23
easy to clean,2013-10-06
easy to clean,2013-06-12


In [23]:
df_word_time4 = df_word_time.filter(df_word_time.col=="easy to install")
display(df_word_time4)

col,reviewTime
easy to install,2015-10-27
easy to install,2015-10-27
easy to install,2014-07-22
easy to install,2013-12-29
easy to install,2015-12-16
easy to install,2015-12-16
easy to install,2016-10-07
easy to install,2016-08-25
easy to install,2016-08-25
easy to install,2015-10-28


In [24]:
df_word_time5 = df_word_time.filter(df_word_time.col=="console game")
display(df_word_time5)

col,reviewTime
console game,2007-12-06
console game,2007-01-14
console game,2014-09-14
console game,2011-12-20
console game,2009-08-11
console game,2008-04-15
console game,2013-04-29
console game,2006-12-28
console game,2009-12-06
console game,2003-06-24


In [25]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

pydf_to_pd = df_preprocess.select("reviewID", "reviewerID", "asin", "joined_column")
pydf_to_pd
df_pd = pydf_to_pd.toPandas()

#SENTIMENT ANALYSIS
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df_pd['reviewTextpolarity'] = df_pd['joined_column'].apply(pol)
df_pd['reviewTextsubjectivity'] = df_pd['joined_column'].apply(sub)
#df_small_pd['summarypolarity'] = df_small_pd['summary'].apply(pol)
#df_small_pd['summarysubjectivity'] = df_small_pd['summary'].apply(sub)

#convert pd df to pyspark df
df_pyspark = spark.createDataFrame(df_pd)

#join sentiment using the 3 IDs
df_final = df_preprocess.join(df_pyspark, (df_preprocess.reviewID == df_pyspark.reviewID) &  (df_preprocess.reviewerID == df_pyspark.reviewerID) & (df_preprocess.asin == df_pyspark.asin), how='left')

/databricks/python/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [26]:

df_cleaned = df_final
df_cleaned.createOrReplaceTempView("df_cleaned")

In [27]:
#Dropping some non required rows to reduce memory requirement.
df_cleaned = df_cleaned.drop('finished_nostopwords','finished_stem', 'finised_lemmat')

In [28]:
#Number of distinct words per category 
from pyspark.sql.functions import countDistinct
df_cleaned.groupBy('type').agg(countDistinct('finished_normalized_wCaps')).show()

+-----+-----------------------------------------+
 type|count(DISTINCT finished_normalized_wCaps)|
+-----+-----------------------------------------+
games| 4529|
books| 9741|
 home| 19217|
+-----+-----------------------------------------+

In [29]:
#Statistics on the number of words (min, max ,avg) per review
df_cleaned.groupBy('type').max("WordCount").show()
df_cleaned.groupBy('type').min("WordCount").show()
df_cleaned.groupBy('type').avg("WordCount").show()

+-----+--------------+
 type|max(WordCount)|
+-----+--------------+
games| 2931|
books| 2986|
 home| 1659|
+-----+--------------+

+-----+--------------+
 type|min(WordCount)|
+-----+--------------+
games| 2|
books| 2|
 home| 2|
+-----+--------------+

+-----+------------------+
 type| avg(WordCount)|
+-----+------------------+
games|132.22882242208743|
books|110.32727820152549|
 home| 54.70977744436109|
+-----+------------------+

###Create Table for tableau

In [31]:
#data table which includes tokens
word_eda = spark.sql("Select type as Category, overall as Rating, vote as HelpfulVote, verified as Verified, label as Useful, reviewTime as ReviewTime, reviewTextsubjectivity as Subjectivity, finished_ngrams as Tokens_Normalized, reviewTextpolarity as Polarity from df_cleaned")

In [32]:
word_eda.write.format("parquet").saveAsTable("GMMA2021_new_york.df_eda_word_final")

In [33]:
#Simplified EDA table without Tokens
eda = spark.sql("Select type as Category, overall as Rating, vote as HelpfulVote, verified as Verified, label as Useful, reviewTime as ReviewTime, reviewTextpolarity as Polarity, reviewTextsubjectivity as Subjectivity, wordCount_normalized as WordCountNormalized, wordCount as WordCount from df_cleaned")


In [34]:
eda.write.format("parquet").saveAsTable("GMMA2021_new_york.df_eda_final")